In [2]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels as stat
import pandas as pd
from pandas.plotting import scatter_matrix
import scipy.stats as stats
import statsmodels.api as sm
import pandas.plotting as pd_plot
import scipy as sc
from pandas import DataFrame
import statsmodels.api as sm

In [3]:
def mean_CI_data(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, std = np.mean(a), np.std(a)
    #h = se * sc.stats.t.ppf((1 + confidence) / 2., n - 1)
    h = std * 1.96 / np.sqrt(n-1)
    return m, m - h, m + h

## Exercise 5.3
We consider data on outdoor running world records for men and women for distances
between 1 km and marathon (42.195 km). The data file worldrecords.dta contain
the world records (as of February 19, 2013) for men and women on 13 different distances
(1=males, 2=females). From 1 km to 10 km the records are set on a running track, while
longer records on the longer distances are set on roads. 

## Part A
### A.1) Fit a linear regression model of time on distance, separately for men and women. Discuss the validity of the model assumptions.

In [8]:
%matplotlib inline

run_data = pd.read_csv('data/worldrecords.csv', sep=';', decimal=",")

run_data_male = run_data.loc[danfund['sex'] == 'Men']
run_data_female = run_data.loc[danfund['sex'] == 'Women']

plt.rcParams.update({'font.size': 13})
fig, ax = plt.subplots(1, 2)

ax[0].scatter(danfund_male['height'], danfund_male['fvc'], color='C0', alpha=0.6, label='male')
ax[0].plot(danfund_male['height'], model_male.predict(sm.add_constant(danfund_male['height'])), color='blue')
ax[0].scatter(danfund_female['height'], danfund_female['fvc'], color='C1', alpha=0.6, label='female')
ax[0].plot(danfund_female['height'], model_female.predict(sm.add_constant(danfund_female['height'])), color='red')
ax[0].set_ylabel('FVC')
ax[0].set_xlabel('height [cm]')
ax[0].legend()

ax[1].scatter(danfund_male['height'], Residuals_male, color='C0', alpha=0.6, label='male')
ax[1].plot(danfund_male['height'], model_male_res.predict(sm.add_constant(danfund_male['height'])), color='blue')
ax[1].scatter(danfund_female['height'], Residuals_female, color='C1', alpha=0.6, label='female')
ax[1].plot(danfund_female['height'], model_female_res.predict(sm.add_constant(danfund_female['height'])), color='red')
ax[1].set_ylabel('Residuals')
ax[1].set_xlabel('height [cm]')
plt.tight_layout()
plt.show()

### A.b) Estimate and interpret the slope for men and women, respectively.

### A.c) Examine if the absolute time difference between men and women is the same for all distances.

### A.d) Predict the expected running time for a world record on 4 km for both men and women

## Part B
### B.1) Fit a linear regression model of log-time on log-distance, separately for men and women. Discuss the validity of the model assumptions.

### B.2) Examine if the relative time difference between men and women is the same for all distances (the hypothesis of no interaction between distance and sex in the model of logtime). 
Use the model of log-time on log-distance with no interaction between distance and sex. 

## Part C

### C.1) Predict the expected running time for a world record on 4 km for both men and women (you may need to use that log(4) = 1.386294). 

### C.2)  Write an equation for the running times as a function of sex and distance. 

### C.3) Quantify the difference between male and female running times.

One parameter predicting the running time is the maximal oxygen consumption
(VO2max), which in a random sample usually follows a normal distribution. In a random
sample of 50 active men and 50 active women the VO2max was found to be an average
of 45.2 (sd=5.1) for the men and 40.4 (sd=4.9) for women. One VO2max was missing
among the men and 4 were missing among the women. 

### C.4) Estimate the absolute mean difference in VO2max between men and women. Estimate without confidence interval the relative mean difference in VO2max between men and women. Do the data suggest the rate of missing data depends on sex? 

### C.5) Discuss if the difference in running times between men and women can be explained by a difference in VO2max.